In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.1-8b-instant"
)

# create tool


In [4]:
from langchain_core.tools import tool

@tool
def multiply(a:int,b:int)-> int:
    """mulitpy of two number"""
    return a * b

In [5]:
print(multiply.invoke({'a': 34, 'b': 45}))

1530


In [6]:
print(multiply.args)

{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


# tool binding


In [7]:
# tool binding

llm_with_tool =llm.bind_tools([multiply])
llm_with_tool

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002A61D133880>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002A61D179CF0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'mulitpy of two number', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

## tool_colling

In [8]:
llm_with_tool.invoke("who are you")

AIMessage(content='I am an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 221, 'total_tokens': 244, 'completion_time': 0.024218077, 'prompt_time': 0.012142888, 'queue_time': 0.050923262, 'total_time': 0.036360965}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a06bee59-a761-423b-931f-51eca2647702-0', usage_metadata={'input_tokens': 221, 'output_tokens': 23, 'total_tokens': 244})

In [9]:
# meatain the message history or chat history
from langchain_core.messages import HumanMessage
queary = HumanMessage("can you solve the question 2 mulitiply by 6")

In [10]:
messagees = [queary]

In [11]:
messagees

[HumanMessage(content='can you solve the question 2 mulitiply by 6', additional_kwargs={}, response_metadata={})]

In [12]:
reslut = llm_with_tool.invoke(messagees) # content is empty after the AIMessage

In [13]:
messagees.append(reslut)

In [14]:
messagees

[HumanMessage(content='can you solve the question 2 mulitiply by 6', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '9pnmz3f65', 'function': {'arguments': '{"a":2,"b":6}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 232, 'total_tokens': 251, 'completion_time': 0.028137107, 'prompt_time': 0.01261989, 'queue_time': 0.050706099, 'total_time': 0.040756997}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--16816f4b-d4a2-4abd-92ee-4fa6523235d0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 6}, 'id': '9pnmz3f65', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 19, 'total_tokens': 251})]

In [15]:
llm_with_tool.invoke("2 mulitiply by 4").tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 4},
  'id': 'fwqvynwb3',
  'type': 'tool_call'}]

In [16]:
llm_with_tool.invoke("2 mulitiply by 4").tool_calls[0]['name']

'multiply'

In [17]:
llm_with_tool.invoke("2 mulitiply by 4").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 2, 'b': 4},
 'id': 'cqvbg4we2',
 'type': 'tool_call'}

In [18]:
llm_with_tool.invoke("2 mulitiply by 4").tool_calls[0]['args']

{'a': 2, 'b': 4}

## tool Exicution

In [19]:
result = llm_with_tool.invoke("2 mulitiply by 4")
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'pvnb8v4da', 'function': {'arguments': '{"a":2,"b":4}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 226, 'total_tokens': 245, 'completion_time': 0.03264222, 'prompt_time': 0.018719036, 'queue_time': 0.053080144, 'total_time': 0.051361256}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--f53d2bdf-8ccf-415b-99a4-0f1e5e2f0afd-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 4}, 'id': 'pvnb8v4da', 'type': 'tool_call'}] usage_metadata={'input_tokens': 226, 'output_tokens': 19, 'total_tokens': 245}


In [20]:
multiply.invoke(llm_with_tool.invoke("2 mulitiply by 4").tool_calls[0]['args'])

8

In [21]:
res= multiply.invoke({'name': 'multiply',
 'args': {'a': 2, 'b': 4},
 'id': 'jh4pvxbx7',
 'type': 'tool_call'})

In [25]:
res

ToolMessage(content='8', name='multiply', tool_call_id='jh4pvxbx7')

In [22]:
messagees.append(res)

In [23]:
messagees

[HumanMessage(content='can you solve the question 2 mulitiply by 6', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '9pnmz3f65', 'function': {'arguments': '{"a":2,"b":6}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 232, 'total_tokens': 251, 'completion_time': 0.028137107, 'prompt_time': 0.01261989, 'queue_time': 0.050706099, 'total_time': 0.040756997}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--16816f4b-d4a2-4abd-92ee-4fa6523235d0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 6}, 'id': '9pnmz3f65', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 19, 'total_tokens': 251}),
 ToolMessage(content='8', name='multiply', tool_call_id='jh4pvxbx7')]

In [24]:
llm_with_tool.invoke(messagees)

AIMessage(content='The result of 2 multiplied by 6 is 12.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 262, 'total_tokens': 276, 'completion_time': 0.018924916, 'prompt_time': 0.014402881, 'queue_time': 0.049700399, 'total_time': 0.033327797}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0e5ecdaa-e943-4736-b919-451de1fb5fe8-0', usage_metadata={'input_tokens': 262, 'output_tokens': 14, 'total_tokens': 276})